In [1]:
from datetime import datetime
from uuid import uuid4
#from mpi4py import MPI

import numpy as np
import mmap
import os
import cv2
from dateutil import tz
from pathlib import Path


from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.ecephys import LFP, ElectricalSeries
from ndx_events import TTLs
from pynwb.image import ImageSeries

In [2]:
directories = r'A:\B119_chronic_NPIX'
final_dir = r'A:\nwb_B119_chronic_NPIX'
num_channels =384
all_folders= []

for folders in os.listdir(directories):
    if os.path.isdir(os.path.join(directories,folders)) and os.path.basename(folders).startswith('2021-'):
        all_folders.append(folders) 
        session_start_time = datetime.strptime(folders, "%Y-%m-%d_%H-%M-%S") 
        nwbfile = NWBFile(
        session_description="Zebra Finch Recording",  
        identifier=str(uuid4()), 
        session_start_time=session_start_time, 
        session_id="B119_chronic_NPIX", 
        experimenter=[
            "Walter, Gonzalez",
        ], 
        lab="Gonzalez Lab",  
        institution="University of California, San Francisco",
        experiment_description="electrophysiological recording of a zebra finch using neuropixel ",  
        )
        nwbfile
        subject = Subject(
        subject_id="001",
        age="P119D",
        description="Black 119 Zebra Finch",
        species="Taeniopygia guttata castanotis",
        sex="M",
        )

        nwbfile.subject = subject
        subject  
        device = nwbfile.create_device(
        name="Neuropixel ", description="Neuropixels 3a probe", manufacturer="Imec")  
        nwbfile.add_electrode_column(name="label", description="adding each electrode column")

        nshanks = 1
        nchannels_per_shank = 384
        electrode_counter = 0

        for ishank in range(nshanks):
            
            electrode_group = nwbfile.create_electrode_group(
                name="shank{}".format(ishank),
                description="electrode group for shank {}".format(ishank),
                device=device,
                location="HVC brain area",
            )
        
            for ielec in range(nchannels_per_shank):
                nwbfile.add_electrode(
                    group=electrode_group,
                    label="shank{}elec{}".format(ishank, ielec),
                    location="brain area",
                )
                electrode_counter += 1
                all_table_region = nwbfile.create_electrode_table_region(
            region=list(range(electrode_counter)),
            description="all recording electrodes",
                )   
        #add ap acquisation
        ap_contFile = os.path.join(directories,folders, 'experiment1','recording1','continuous','Neuropix-PXI-100.0','continuous.dat')
        FS = 30000
        num_channels = 384
        file_size = os.path.getsize(ap_contFile)
        samples = file_size / 2 / num_channels
        int_sample = int(samples)
        ap_data = np.memmap(ap_contFile, dtype='int16', mode='r')
        reshape_data = np.reshape(ap_data,(int_sample,384))

        path_to_timestamp = os.path.join(directories,folders,'experiment1','recording1','continuous','Neuropix-PXI-100.0','timestamps.npy')
        timestamps_AP= np.load(path_to_timestamp)
        timestamp_APP=timestamps_AP[0]
        int_sample = int(samples)

        ap = ElectricalSeries(
        name="AP ElectricalSeries",
        data=ap_data,
        electrodes=all_table_region, 
        timestamps= timestamps_AP,
        description ='sampled in 30000 HZ',
                ) 
        nwbfile.add_acquisition(ap)

        #add lfp acquisation
        lfp_contFile = os.path.join(directories,folders, 'experiment1','recording1','continuous','Neuropix-PXI-100.1','continuous.dat')
        FS = 2500
        file_size = os.path.getsize(lfp_contFile)
        samples = file_size / 2 / num_channels
        lfp_data = np.memmap(lfp_contFile, dtype='int16', mode='r')

        path_to_timestamp = os.path.join(directories,folders,'experiment1','recording1','continuous','Neuropix-PXI-100.1','timestamps.npy')
        timestamps_lfp= np.load(path_to_timestamp)
        timestamp_lfpp=timestamps_lfp[0]
        int_sample = int(samples)

        lfp = ElectricalSeries(
        name="Lfp ElectricalSeries",
        data=lfp_data,
        electrodes=all_table_region, 
        timestamps= timestamps_lfp,
        #rate=2500.0, 
        description ='sampled in 2500HZ',
                ) 
        nwbfile.add_acquisition(lfp)


        #Add nidaq acqusation
        nidaq_contFile = os.path.join(directories,folders, 'experiment1','recording1','continuous','NI-DAQmx-105.0','continuous.dat')
        FS = 2500
        num_channels = 384
        file_size = os.path.getsize(nidaq_contFile)
        samples = file_size / 2 / num_channels
        nidaq_data = np.memmap(nidaq_contFile, dtype='int16', mode='r')

        path_to_timestamp = os.path.join(directories,folders,'experiment1','recording1','continuous','NI-DAQmx-105.0','timestamps.npy')
        timestamps_nidaq= np.load(path_to_timestamp)
        timestamp_nidaqq=timestamps_nidaq[0]
        int_sample = int(samples)

        Nidaq = ElectricalSeries(
        name="Nidaq ElectricalSeries",
        data=nidaq_data,
        electrodes=all_table_region, 
        timestamps=timestamps_nidaq,
        #rate=2500.0, 
        description ='sampled in 2500HZ',
                ) 
        nwbfile.add_acquisition(Nidaq)

        #add events acquisation (save TTL pulse data as timeseries)
        events_timestamp = os.path.join(directories,folders,'experiment1','recording1','events','Neuropix-PXI-100.0','TTL_1','timestamps.npy')
        timestamps_all = np.load(events_timestamp)
        sampleRate = 30000
        data_root = os.path.join(directories,folders,'experiment1','recording1','events')

        data_files = {
        'ap_channel_states': np.load(os.path.join(directories,folders,'experiment1','recording1','events','Neuropix-PXI-100.0', 'TTL_1', 'channel_states.npy')),
        'ap_channels': np.load(os.path.join(directories,folders,'experiment1','recording1','events','Neuropix-PXI-100.0', 'TTL_1', 'channels.npy')),
        'ap_full_words': np.load(os.path.join(directories,folders,'experiment1','recording1','events','Neuropix-PXI-100.0', 'TTL_1', 'full_words.npy'))
                }
        for name, data in data_files.items():
            ttlData = TimeSeries(
                name=f'TTL_{name}',
                data=data.astype("uint16"),
                timestamps=timestamps_all,
                unit='NA',
                description=f'AP_TTL {name.replace(".npy", "")}'
            )
            nwbfile.add_acquisition(ttlData)
        
        #nidaq events
        events_timestamp = os.path.join(directories,folders,'experiment1','recording1','events','NI-DAQmx-105.0','TTL_1','timestamps.npy')
        timestamps_all = np.load(events_timestamp)
        sampleRate = 30000
        data_root = os.path.join(directories,folders,'experiment1','recording1','events')

        data_files = {
        'nidaq_channel_states': np.load(os.path.join(directories,folders,'experiment1','recording1','events','NI-DAQmx-105.0', 'TTL_1', 'channel_states.npy')),
        'nidaq_channels': np.load(os.path.join(directories,folders,'experiment1','recording1','events','NI-DAQmx-105.0', 'TTL_1', 'channels.npy')),
        'nidaq_full_words': np.load(os.path.join(directories,folders,'experiment1','recording1','events','NI-DAQmx-105.0', 'TTL_1', 'full_words.npy'))
                }
        for name, data in data_files.items():
            nidaq_ttlData = TimeSeries(
                name=f'TTL_{name}',
                data=data.astype("uint16"),
                timestamps=timestamps_all,
                unit='NA',
                description=f'Nidaq_TTL {name.replace(".npy", "")}'
            )
            nwbfile.add_acquisition(nidaq_ttlData)
        
        #message center events
        events_timestamp = os.path.join(directories,folders,'experiment1','recording1','events','Message_Center-904.0','TEXT_group_1','timestamps.npy')
        timestamps_all = np.load(events_timestamp)
        sampleRate = 30000
        data_root = os.path.join(directories,folders,'experiment1','recording1','events')

        data_files = {
        'msg_channel_states': np.load(os.path.join(directories,folders,'experiment1','recording1','events','Message_Center-904.0','TEXT_group_1','channels.npy')),
        'msg_channels': np.load(os.path.join(directories,folders,'experiment1','recording1','events','Message_Center-904.0','TEXT_group_1','text.npy')),
    
                }
        for name, data in data_files.items():
            msg_ttlData = TimeSeries(
                name=f'TTL_{name}',
                data=data.astype("uint16"),
                timestamps=timestamps_all,
                unit='NA',
                description=f'Nidaq_TTL {name.replace(".npy", "")}'
            )
            nwbfile.add_acquisition(msg_ttlData)
        

        nwbfile_path = os.path.join(final_dir,folders) 
        if not os.path.exists(nwbfile_path):
            os.mkdir(nwbfile_path)
        nwbfile_path = os.path.join(nwbfile_path, 'nwb_file.nwb')
        with NWBHDF5IO(nwbfile_path, "w") as io:
            io.write(nwbfile) 
            
print(len(all_folders))



c:\Users\Michael\.conda\envs\pynwb\Lib\site-packages\pynwb\file.py:471: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)
c:\Users\Michael\.conda\envs\pynwb\Lib\site-packages\pynwb\base.py:193: UserWarning: ElectricalSeries 'AP ElectricalSeries': Length of data does not match length of timestamps. Your data may be transposed. Time should be on the 0th dimension
  warn("%s '%s': Length of data does not match length of timestamps. Your data may be transposed. "
c:\Users\Michael\.conda\envs\pynwb\Lib\site-packages\pynwb\base.py:193: UserWarning: ElectricalSeries 'Lfp ElectricalSeries': Length of data does not match length of timestamps. Your data may be transposed. Time should be on the 0th dimension
  warn("%s '%s': Length of data does not match length of timestamps. Your data may be transposed. "
c:\Users\Michael\.conda\envs\pynwb\Lib\site-packages\pynwb\base.py:193: UserWarning:

In [ ]:
nwbfile

In [11]:
#test the nwb file
nwbfile_path = r'D:\testing_first\2021-04-16_21-46-47\nwb_file.nwb'
io = NWBHDF5IO(nwbfile_path, mode='r',load_namespaces=True)
nwbfile= io.read()
nwbfile

root pynwb.file.NWBFile at 0x2308868458256
Fields:
  acquisition: {
    Nidaq ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    Neuropixel  <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    shank0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: electrophysiological recording of a zebra finch using neuropixel 
  experimenter: ['Walter, Gonzalez']
  file_create_date: [datetime.datetime(2024, 1, 9, 16, 29, 13, 574491, tzinfo=tzoffset(None, -28800))]
  identifier: 345a220c-f8e3-4c27-9dff-f1329b9debc7
  institution: University of California, San Francisco
  lab: Gonzalez Lab
  session_description: Zebra Finch Recording
  session_id: B119_chronic_NPIX
  session_start_time: 2021-04-16 21:46:47-07:00
  subject: subject pynwb.file.Subject at 0x2310471584272
Fields:
  age: P60D
  age__reference: birth
  description: Black 119  Zebra Finch
  sex: M
  species: Taeniopygia guttata castanotis
  subject_id: 001

  timestamps_reference_time: 2021-04-16 21:46:47-07:00

In [13]:
nwbfile.acquisition

{'Nidaq ElectricalSeries': Nidaq ElectricalSeries pynwb.ecephys.ElectricalSeries at 0x2310479319888
 Fields:
   comments: no comments
   conversion: 1.0
   description: sampled in 2500HZ
   offset: 0.0
   rate: 2500.0
   resolution: -1.0
   starting_time: 0.0
   starting_time_unit: seconds
   unit: volts}

In [14]:
io.close()